# Change Numerical Data Distributions 

Adapted from Jason Brownlee. 2020. [Data Preparation for Machine Learning](https://machinelearningmastery.com/data-preparation-for-machine-learning/).

## Overview

Numerical input variables may have a highly skewed or non-standard distribution. This could be
caused by outliers in the data, multi-modal distributions, highly exponential distributions, and
more. Many machine learning algorithms prefer or perform better when numerical input variables
and even output variables in the case of regression have a standard probability distribution,
such as a Gaussian (normal) or a Uniform distribution.

The quantile transform provides an automatic way to transform a numeric input variable to
have a different data distribution, which in turn, can be used as input to a predictive model.

## Learning Objectives

- Learn why machine learning algorithms prefer numerical variables with Gaussian or standard probability distributions
- Understand quantile transforms for transforming numerical variables to Gaussian or Uniform distributions 
- Learn how to use `QuantileTransformer` to change probability distributions of numeric variables to improve model performance

### Tasks

- Apply normal quantile transform
- Apply uniform quantile transform  
- Compare model performance with different transforms
- Explore impact of quantile parameters

## Prerequisites

- Basic understanding of Python programming
- Familiarity with NumPy libraries
- Knowledge of basic statistical concepts

## Get Started

To start, we install required packages, import the necessary libraries.

### Install required packages

In [ ]:
# This line uses the pip package installer to install the specified Python libraries if they are not already installed.
# matplotlib: A comprehensive library for creating static, interactive, and animated visualizations in Python.
# numpy: NumPy is the fundamental package for scientific computing in Python. It is used for working with arrays.
# pandas: pandas is a powerful data manipulation and analysis library. It provides data structures like DataFrames.
# scikit-learn: scikit-learn is a machine learning library that provides tools for classification, regression, clustering, dimensionality reduction, model selection, and preprocessing.
%pip install matplotlib numpy pandas scikit-learn

### Import necessary libraries

In [ ]:
# Import the pyplot module from matplotlib library and alias it as plt for plotting functionalities.
import matplotlib.pyplot as plt
# Import specific functions (exp, mean, std) from the numpy library for numerical operations.
from numpy import exp, mean, std
# Import the randn function from numpy.random module for generating random numbers from a standard normal distribution.
from numpy.random import randn
# Import DataFrame and read_csv functions from the pandas library for data manipulation and reading CSV files.
from pandas import DataFrame, read_csv
# Import RepeatedStratifiedKFold and cross_val_score from sklearn.model_selection for model evaluation using cross-validation.
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
# Import KNeighborsClassifier from sklearn.neighbors for using the K-Nearest Neighbors classification algorithm.
from sklearn.neighbors import KNeighborsClassifier
# Import Pipeline from sklearn.pipeline for creating a pipeline to chain preprocessing and model steps.
from sklearn.pipeline import Pipeline
# Import LabelEncoder and QuantileTransformer from sklearn.preprocessing for data preprocessing tasks.
from sklearn.preprocessing import LabelEncoder, QuantileTransformer

# Define a variable 'pima_indians_diabetes_csv' that stores the file path to the Pima Indians Diabetes dataset CSV file.
pima_indians_diabetes_csv = "../../Data/pima-indians-diabetes.csv"

##  Quantile Transforms

A quantile transform will map a variable's probability distribution to another probability
distribution. The quantile function ranks or smooths out the relationship between observations and can be
mapped onto other distributions, such as the uniform or normal distribution. The transformation
can be applied to each numeric input variable in the training dataset and then provided as
input to a machine learning model to learn a predictive modeling task. This quantile transform
is available in the scikit-learn Python machine learning library via the **QuantileTransformer**
class.

We first creates a sample of 1,000 random Gaussian values and adds a
skew to the dataset. A histogram is created from the skewed dataset and clearly shows the
distribution pushed to the far left. 



In [ ]:
# demonstration of the quantile transform

# generate gaussian data sample
# Generate 1000 random numbers from a standard normal distribution (Gaussian distribution) using randn function.
data = randn(1000)

# add a skew to the data distribution
# Apply the exponential function to each data point to introduce a right skew in the distribution.
# Exponential transformation makes positive values larger and compresses negative values towards zero, creating skewness.
data = exp(data)

# histogram of the raw data with a skew
# Create a histogram plot of the transformed 'data' to visualize its distribution.
# 'bins=25' specifies that the histogram should have 25 bins.
plt.hist(data, bins=25)
# Display the histogram plot.
plt.show()

Then a QuantileTransformer is used to map the data to a Gaussian distribution and standardize the result, centering the values on the mean value of 0 and a standard deviation of
1.0. A histogram of the transform data is created showing a Gaussian shaped data distribution.

In [ ]:
# Reshape the 'data' array to have rows and a single column.
# This is often necessary for scikit-learn transformers that expect 2D input.
data = data.reshape((len(data), 1))
# Print the shape of the reshaped 'data' array.
# This will output the new dimensions, confirming it's now a 2D array with one column.
print(data.shape)

# Initialize a QuantileTransformer object with the output distribution set to 'normal'.
# 'output_distribution="normal"' specifies that the transformed data should follow a normal (Gaussian) distribution as closely as possible.
quantile = QuantileTransformer(output_distribution="normal")
# Fit the QuantileTransformer to the 'data' and then transform 'data'.
# 'fit_transform' learns the quantile information from 'data' and applies the normal quantile transformation.
data_trans = quantile.fit_transform(data)

# Create a histogram of the transformed data 'data_trans'.
# plt.hist(data_trans, bins=25) generates a histogram with 25 bins to visualize the distribution of the transformed data.
plt.hist(data_trans, bins=25)
# Display the histogram plot.
# plt.show() command is used to open a window and display the generated plot.
plt.show()

## Diabetes Dataset

The dataset classifies patient data as
either an onset of diabetes within five years or not. 

```
Number of Instances: 768
Number of Attributes: 8 plus class 
For Each Attribute: (all numeric-valued)
   1. Number of times pregnant
   2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
   3. Diastolic blood pressure (mm Hg)
   4. Triceps skin fold thickness (mm)
   5. 2-Hour serum insulin (mu U/ml)
   6. Body mass index (weight in kg/(height in m)^2)
   7. Diabetes pedigree function
   8. Age (years)
   9. Class variable (0 or 1)
Missing Attribute Values: Yes
Class Distribution: (class value 1 is interpreted as "tested positive for
   diabetes")
   Class Value  Number of instances
   0            500
   1            268
```

You can learn more about the dataset here:

* Diabetes Dataset File ([pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv))
* Diabetes Dataset Details ([pima-indians-diabetes.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names))

### Summarizing the variables from the pima-indians-diabetes dataset

In [ ]:
# load and summarize the dataset

# load dataset from a CSV file named 'pima_indians_diabetes_csv' into a pandas DataFrame.
# 'header=None' argument indicates that the CSV file does not have a header row.
dataset = read_csv(pima_indians_diabetes_csv, header=None)
# Print the first few rows of the DataFrame to get a glimpse of the data.
# By default, head() displays the first 5 rows.
print(dataset.head())

# summarize the shape of the dataset
# Print the shape of the DataFrame, which includes the number of rows and columns.
print(dataset.shape)

# summarize each variable (column) in the dataset
# Print descriptive statistics for each column in the DataFrame.
# This includes count, mean, std, min, 25%, 50%, 75%, max for numerical columns.
print(dataset.describe())

This confirms the 8
input variables, one output variable, and 768 rows of data. 

Finally a histogram is created for each input variable. If we ignore the clutter of the plots and
focus on the histograms themselves, we can see that many variables have a skewed distribution.
The dataset provides a good candidate for using a quantile transform to make the variables
more-Gaussian.

In [ ]:
# Create histograms for each variable in the 'dataset' DataFrame.
# 'dataset.hist()' is a pandas function that generates histograms for all numerical columns in the DataFrame.
# 'xlabelsize=4' and 'ylabelsize=4' arguments set the font size of the x and y axis labels to 4 points, making them smaller.
fig = dataset.hist(xlabelsize=4, ylabelsize=4)

# Iterate over each subplot (histogram) in the figure returned by 'dataset.hist()'.
# 'fig' is a NumPy array of matplotlib AxesSubplot objects, where each subplot is a histogram for a column.
# 'fig.ravel()' flattens this array into a 1D array for easy iteration.
[x.title.set_size(4) for x in fig.ravel()]
# For each subplot 'x', this line sets the title font size to 4 points, making the titles smaller.
# 'x.title' accesses the title object of the subplot.
# 'set_size(4)' sets the font size of the title.

# Display the plot.
# 'plt.show()' is a matplotlib function that opens a window and displays all generated figures.
plt.show()

Next, let's first and evaluate a machine learning model on the raw dataset. We will use
a k-nearest neighbor algorithm with default hyperparameters and evaluate it using repeated
stratified k-fold cross-validation.

In [ ]:
# evaluate KNN classifier on the raw dataset

# KFold is a cross-validator that divides the dataset into k folds.
# Stratified is to ensure that each fold of dataset has the same proportion of
# observations with a given label.
# Repeated provides a way to improve the estimated performance of a machine
# learning model.
# This involves simply repeating the cross-validation procedure multiple times
# and reporting the mean result across all folds from all runs. This mean result
# is expected to be a more accurate estimate of the true unknown underlying mean
# performance of the model on the dataset, as calculated using the standard
# error.

# load dataset
# Load the dataset from the 'pima_indians_diabetes.csv' file using pandas' read_csv function.
# 'header=None' indicates that the CSV file does not have a header row.
dataset = read_csv(pima_indians_diabetes_csv, header=None)
# Extract the values from the pandas DataFrame into a NumPy array.
data = dataset.values

# separate into input and output columns
# Separate the dataset into input features (X) and output labels (y).
# X is assigned all columns except the last one ([:-1]).
# y is assigned the last column ([-1]), which is assumed to be the target variable.
X, y = data[:, :-1], data[:, -1]

# ensure inputs are floats and output is an integer label
# Ensure that the input features (X) are of float32 data type.
X = X.astype("float32")
# Encode the output labels (y) into integer labels using LabelEncoder.
# fit_transform fits the LabelEncoder to y and then transforms y to numerical labels.
y = LabelEncoder().fit_transform(y.astype("str"))

# define and configure the model using
# Classifier implementing the k-nearest neighbors vote.
# Define the KNN classifier model.
# KNeighborsClassifier() creates a KNN classifier object with default parameters.
model = KNeighborsClassifier()

# evaluate the model using RepeatedStratifiedKFold cross validator,
# that repeats Stratified K-Fold n times with different randomization in each
# repetition.
# Define the cross-validation strategy using RepeatedStratifiedKFold.
# n_splits=10:  Divides the dataset into 10 folds.
# n_repeats=3: Repeats the cross-validation process 3 times.
# random_state=1: Sets the random seed for reproducibility.
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# Evaluate the model using cross-validation.
# model: The KNN classifier model to evaluate.
# X: The input features.
# y: The output labels.
# scoring='accuracy': The metric used to evaluate the model (accuracy in this case).
# cv: The cross-validation strategy defined above (RepeatedStratifiedKFold).
# n_jobs=-1: Use all available CPU cores for parallel execution.
n_scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# report model performance
# Print the model's performance.
# mean(n_scores): Calculates the mean accuracy across all folds and repeats.
# std(n_scores): Calculates the standard deviation of the accuracy scores.
# "Accuracy: %.3f (%.3f)": Formats the output string to display the mean and standard deviation rounded to 3 decimal places.
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

In this case we can see that the model achieved a mean classification accuracy of about 71.7
percent.

## Normal Quantile Transform

It is often desirable to transform an input variable to have a normal probability distribution to improve the modeling performance. We can apply the Quantile transform using the
QuantileTransformer class and set the output distribution argument to `normal'. We
must also set the n quantiles argument to a value less than the number of observations in the
training dataset, in this case, 100. Once defined, we can call the fit transform() function and
pass it to our dataset to create a quantile transformed version of our dataset.

In [ ]:
# visualize a normal quantile transform of the dataset

# load dataset
# Load the Pima Indians Diabetes dataset from the CSV file 'pima_indians_diabetes_csv' into a pandas DataFrame.
# 'header=None' indicates that the CSV file does not have a header row.
dataset = read_csv(pima_indians_diabetes_csv, header=None)

# retrieve just the numeric input values
# Extract the numpy array of values from the DataFrame and select all rows (:) and all columns except the last one (:-1).
# This assumes the last column is the target variable and the preceding columns are input features.
data = dataset.values[:, :-1]

# perform a normal quantile transform of the dataset 'n_quantiles" is the number
# of quantiles to be computed. It corresponds to the number of landmarks used to
# discretize the cumulative distribution function.  'output_distribution" is the
# marginal distribution for the transformed data. The choices are 'uniform'
# (default) or 'normal'.
# Initialize a QuantileTransformer object.
# 'n_quantiles=100' sets the number of quantiles to 100, controlling the granularity of the transformation.
# 'output_distribution="normal"' specifies that the transformed data should follow a normal distribution.
trans = QuantileTransformer(n_quantiles=100, output_distribution="normal")
# Fit the QuantileTransformer to the input data and then transform the data.
# 'fit_transform' learns the quantile mapping from the data and applies the transformation.
data = trans.fit_transform(data)

# convert the array back to a dataframe
# Convert the transformed numpy array 'data' back into a pandas DataFrame.
dataset = DataFrame(data)

# histograms of the variables
# Create histograms for each variable (column) in the DataFrame.
# 'xlabelsize=4' and 'ylabelsize=4' set the font size of the x and y axis labels to 4 for better readability in the plot.
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
# Iterate over each histogram subplot in the figure and set the title font size to 4.
# 'fig.ravel()' flattens the array of subplots into a 1D array.
# 'x.title.set_size(4)' sets the font size of the title for each subplot.
[x.title.set_size(4) for x in fig.ravel()]

# show the plot
# Display the generated histograms plot.
plt.show()

We can see that the shape of the histograms for each variable looks very Gaussian as compared
to the raw data.

Next, let's evaluate the same KNN model as the previous section, but in this case on a
normal quantile transform of the dataset

In [ ]:
# Evaluate KNN with normal quantile transform

# load dataset from the specified CSV file 'pima_indians_diabetes_csv' without a header row.
dataset = read_csv(pima_indians_diabetes_csv, header=None)
# Extract the values from the pandas DataFrame into a NumPy array.
data = dataset.values

# separate the data into input features (X) and output target (y) columns.
# X gets all columns except the last one ([:-1]).
# y gets only the last column ([:, -1]).
X, y = data[:, :-1], data[:, -1]

# ensure inputs (X) are of float32 data type for numerical operations.
X = X.astype("float32")
# encode the output labels (y) into integer labels using LabelEncoder.
# fit_transform fits the LabelEncoder to y and then transforms y to encoded labels.
y = LabelEncoder().fit_transform(y.astype("str"))

# define the quantile transformer to transform features to a normal distribution.
# n_quantiles=100 specifies the number of quantiles to use in the transformation.
# output_distribution="normal" specifies that the output should be a normal distribution.
trans = QuantileTransformer(n_quantiles=100, output_distribution="normal")

# define the model, using KNeighborsClassifier which is a k-nearest neighbors classifier.
model = KNeighborsClassifier()
# create a pipeline that first applies the quantile transformation (trans) and then the KNN model (model).
# steps=[("t", trans), ("m", model)] defines the steps in the pipeline with names 't' for transformer and 'm' for model.
pipeline = Pipeline(steps=[("t", trans), ("m", model)])

# define cross-validation configuration using RepeatedStratifiedKFold.
# n_splits=10 means 10 folds for cross-validation.
# n_repeats=3 means repeat cross-validation 3 times.
# random_state=1 ensures reproducibility of the cross-validation splits.
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the pipeline using cross_validation.
# pipeline is the defined pipeline to evaluate.
# X is the input data, y is the target data.
# scoring="accuracy" specifies that accuracy is the metric to evaluate.
# cv=cv uses the defined RepeatedStratifiedKFold for cross-validation.
# n_jobs=-1 uses all available CPU cores for parallel processing to speed up cross-validation.
n_scores = cross_val_score(pipeline, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# report the pipeline performance.
# mean(n_scores) calculates the average accuracy across all cross-validation folds and repeats.
# std(n_scores) calculates the standard deviation of the accuracy scores.
# "Accuracy: %.3f (%.3f)" is a format string to print the mean and standard deviation, rounded to 3 decimal places.
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

We can see that the normal quantile transform results in a lift in
performance from 71.7 percent accuracy without the transform to about 73.4 percent with the
transform.

## Uniform Quantile Transform

Sometimes it can be beneficial to transform a highly exponential or multi-modal distribution to
have a uniform distribution. This is especially useful for data with a large and sparse range of
values, e.g. outliers that are common rather than rare. We can apply the transform by defining
a QuantileTransformer class and setting the output distribution argument to `uniform'
(the default).

In [ ]:
# visualize a uniform quantile transform of the dataset

# load dataset
# Load the Pima Indians Diabetes dataset from a CSV file named 'pima_indians_diabetes_csv' into a pandas DataFrame.
# 'header=None' indicates that the CSV file has no header row.
dataset = read_csv(pima_indians_diabetes_csv, header=None)

# retrieve just the numeric input values
# Extract all rows and all columns except the last one (which is assumed to be the target variable).
# ':,' selects all rows, and ':-1' selects all columns up to, but not including, the last column.
data = dataset.values[:, :-1]

# perform a uniform quantile transform of the dataset
# Initialize a QuantileTransformer object.
# 'n_quantiles=100' specifies that the data should be mapped to 100 quantiles.
# 'output_distribution="uniform"' sets the output distribution to be uniform, meaning values will be spread evenly between 0 and 1.
trans = QuantileTransformer(n_quantiles=100, output_distribution="uniform")
# Fit the QuantileTransformer to the data and then transform the data.
# 'fit_transform' learns the quantile mapping from the input data and applies the transformation.
data = trans.fit_transform(data)

# convert the array back to a dataframe
# Convert the NumPy array 'data' back into a pandas DataFrame.
# This is done to easily use pandas' plotting capabilities for histograms.
dataset = DataFrame(data)

# histograms of the variables
# Create histograms for each column (variable) in the DataFrame.
# 'xlabelsize=4' and 'ylabelsize=4' set the font size of the x and y axis labels to 4.
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
# Reduce the size of the titles of each histogram subplot.
# 'fig.ravel()' flattens the array of subplots returned by 'dataset.hist()'.
# '[x.title.set_size(4) for x in fig.ravel()]' iterates through each subplot and sets the title size to 4.
[x.title.set_size(4) for x in fig.ravel()]

# show the plot
# Display the generated histograms plot.
plt.show()

We can see that the shape of the histograms for each variable looks very uniform compared to
the raw data.

Next, let's evaluate the same KNN model as the previous section, but in this case on a
uniform quantile transform of the raw dataset.

In [ ]:
# evaluate KNN classifier on the dataset with uniform quantile transform

# load dataset
# Load the Pima Indians Diabetes dataset from a CSV file named 'pima_indians_diabetes_csv' using pandas.
# 'header=None' indicates that the CSV file does not have a header row.
dataset = read_csv(pima_indians_diabetes_csv, header=None)
# Extract the values from the pandas DataFrame and store them in a NumPy array called 'data'.
data = dataset.values

# separate into input and output columns
# Separate the dataset into input features (X) and output labels (y).
# X is assigned all columns except the last one ([:-1]).
# y is assigned the last column ([-1]).
X, y = data[:, :-1], data[:, -1]

# ensure inputs are floats and output is an integer label
# Ensure that the input features (X) are of float32 data type.
X = X.astype("float32")
# Encode the output labels (y) into integer labels using LabelEncoder.
# 'astype("str")' is used to handle potential mixed data types in the label column before encoding.
y = LabelEncoder().fit_transform(y.astype("str"))

# define the pipeline
# Define a QuantileTransformer for preprocessing.
# 'n_quantiles=100' specifies the number of quantiles to use for transformation.
# 'output_distribution="uniform"' sets the output distribution to be uniform after transformation.
trans = QuantileTransformer(n_quantiles=100, output_distribution="uniform")
# Define a KNeighborsClassifier model.
model = KNeighborsClassifier()
# Create a pipeline that first applies the QuantileTransformer (trans) and then the KNeighborsClassifier (model).
# The steps are named 't' for transformer and 'm' for model.
pipeline = Pipeline(steps=[("t", trans), ("m", model)])

# evaluate the pipeline
# Define a RepeatedStratifiedKFold cross-validation strategy.
# 'n_splits=10' specifies 10 folds for each cross-validation iteration.
# 'n_repeats=3' specifies to repeat the cross-validation 3 times.
# 'random_state=1' ensures reproducibility of the shuffling in cross-validation.
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# Evaluate the pipeline using cross-validation.
# 'pipeline' is the estimator to evaluate.
# 'X' is the input data.
# 'y' is the target data.
# 'scoring="accuracy"' specifies that accuracy is the metric to evaluate.
# 'cv=cv' uses the RepeatedStratifiedKFold cross-validation strategy defined earlier.
# 'n_jobs=-1' uses all available CPU cores for parallel processing.
n_scores = cross_val_score(pipeline, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# report pipeline performance
# Report the performance of the pipeline.
# 'mean(n_scores)' calculates the average accuracy across all cross-validation folds and repeats.
# 'std(n_scores)' calculates the standard deviation of the accuracy scores.
# "Accuracy: %.3f (%.3f)" is a format string to print the mean and standard deviation, rounded to 3 decimal places.
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

We can see that the uniform transform results in a lift in performance
from 71.7 percent accuracy without the transform to about 73.4 percent with the normal transform, and achieved a score of 74.1 percent.

We chose the number of quantiles as an arbitrary number, in this case, 100. This hyperparameter can be tuned to explore the effect of the resolution of the transform on the resulting
skill of the model. The example below performs this experiment and plots the mean accuracy
for different n quantiles values from 1 to 99.

In [ ]:
# explore number of quantiles on classification accuracy

# Define a function named 'get_dataset' that takes a 'filename' as input.
def get_dataset(filename):
    # Load the dataset from a CSV file specified by 'filename' into a pandas DataFrame.
    # 'header=None' argument indicates that the CSV file does not have a header row.
    dataset = read_csv(filename, header=None)
    
    # Convert the pandas DataFrame 'dataset' into a NumPy array 'data'.
    data = dataset.values
    
    # Split the 'data' array into input features (X) and output labels (y).
    # '[:, :-1]' selects all rows and all columns except the last one for input features (X).
    # '[:, -1]' selects all rows and only the last column for output labels (y).
    X, y = data[:, :-1], data[:, -1]
    
    # Convert the input features 'X' to float32 data type.
    X = X.astype("float32")
    
    # Encode the output labels 'y' using LabelEncoder to convert string labels to numerical labels.
    y = LabelEncoder().fit_transform(y.astype("str"))
    
    # Return the processed input features 'X' and output labels 'y'.
    return X, y


# Define a function named 'get_models' to create and return a dictionary of machine learning models (pipelines).
def get_models():
    # Initialize an empty dictionary called 'models' to store the pipelines, where keys will be the number of quantiles and values will be the pipelines themselves.
    models = dict()
    
    # Loop through numbers from 1 to 99 (inclusive), where each number 'i' represents the number of quantiles for the QuantileTransformer.
    for i in range(1, 100):
        # define the pipeline
        # Create a QuantileTransformer object named 'trans' to transform the input features using quantiles.
        trans = QuantileTransformer(n_quantiles=i, output_distribution="uniform")
        
        # Create a KNeighborsClassifier object named 'model' which will be used as the classification model in the pipeline.
        model = KNeighborsClassifier()
        
        # The purpose of the pipeline is to assemble several steps that can be
        # cross-validated together while setting different parameters.
        # Create a Pipeline object that chains together the QuantileTransformer ('trans') and KNeighborsClassifier ('model').
        # The pipeline is stored in the 'models' dictionary with the number of quantiles 'i' as the key (converted to a string).
        models[str(i)] = Pipeline(steps=[("t", trans), ("m", model)])
        
    # Return the 'models' dictionary containing all the created pipelines.
    return models


# Define a function named 'evaluate_model' that takes a machine learning 'model', input features 'X', and output labels 'y' as input.
def evaluate_model(model, X, y):
    # Create a RepeatedStratifiedKFold cross-validation object named 'cv'.
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    
    # Perform cross-validation using 'cross_val_score' to evaluate the 'model'.
    scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)
    
    # Return the list of accuracy scores obtained from cross-validation.
    return scores

# define dataset
# Call the get_dataset function to load and prepare the dataset, and unpack the features (X) and labels (y)
X, y = get_dataset(pima_indians_diabetes_csv)

# get the models to evaluate
# Call the function `get_models()` to retrieve a dictionary of machine learning models that will be evaluated.
models = get_models()

# Initialize an empty list called 'results' to store the mean accuracy scores for each model.
results = list()
# Start a loop that iterates through the 'models' dictionary. 'models.items()' returns key-value pairs (name, model).
for name, model in models.items():
    # Evaluate the current 'model' using the 'evaluate_model' function with the dataset (X, y).
    # The 'evaluate_model' function performs cross-validation and returns a list of accuracy scores.
    scores = evaluate_model(model, X, y)
    
    # Calculate the mean of the 'scores' list (which represents the average accuracy from cross-validation) and append it to the 'results' list.
    results.append(mean(scores))
    
    # Print the performance of the current model.
    # '>%s %.3f (%.3f)' is a format string:
    #   - '%s' will be replaced by the 'name' of the model (which is the number of quantiles in this case).
    #   - '%.3f' will be replaced by the mean accuracy, formatted to 3 decimal places.
    #   - '%.3f' will be replaced by the standard deviation of the accuracy scores, formatted to 3 decimal places.
    print(">%s %.3f (%.3f)" % (name, mean(scores), std(scores)))

In this case, we can see that surprisingly smaller values resulted in better accuracy, with
values such as 9 achieving an accuracy of about 74.7 percent.

In [ ]:
# plot model performance for comparison
# Plot the 'results' data. Assuming 'results' is a list or array of performance metrics (e.g., accuracy scores) from different models or configurations.
plt.plot(results)
# Display the plot created by plt.plot(). This will open a window showing the graph.
plt.show()

A line plot is created showing the number of quantiles used in the transform versus the
classification accuracy of the resulting model. We can see a bump with values less than 10 and
drop and 
at performance after that. The results highlight that there is likely some benefit in
exploring different distributions and number of quantiles to see if better performance can be
achieved.

## Conclusion

In this tutorial, we explored the transformation of numerical input variables that exhibit skewed or non-standard distributions due to factors like outliers, multi-modal patterns, or exponential distributions. Then we explored the performance of machine learning algorithms both with and without transformation. 

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.